# 신경망 시작하기

## 신경망의 구조

- LSTM과 같은 순환 층 : (samples, timesteps, features) 크기의 3D 텐서로 저장된 시퀀스 데이터 처리
- 2D 합성곱(Conv2D) : 4D 텐서로 저장되어 있는 이미지 데이터 처리

In [4]:
# 층 호환성 : 각 층은 특정 크기의 입력 텐서만 받고 특정 크기의 출력 텐서를 반환한다.
from keras import layers
layer = layers.Dense(32, input_shape=(784,)) # 32개의 유닛으로 된 밀집 층

In [6]:
# 케라스에서는 모델에 추가된 층을 자동으로 상위 층의 크기에 맞추어 주기 때문에 호환성 걱정을 하지 않아도 된다.
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.Dense(10))  # input_shape를 따로 지정하지 않았지만, 앞선 층의 출력 크기를 입력 크기로 자동으로 채택.

- 용도별 손실함수
- binary crossentropy : 2개의 클래스가 있는 분류 문제
- categorical crossentropy : 여러 개의 클래스가 있는 분류 문제
- MSE : 회귀문제
- CTC(Connection Temporal Classification) : 시퀀스 학습 문제
- └─> 음성 인식이나 필기 인식처럼 입력에 레이블 할당 위치를 정하기 어려운 연속적인 시퀀스를 다루는 문제에 사용

## 케라스 소개

- 코드 예제를 위해 [케라스](http://keras.io)를 사용.
- 동일한 코드로 CPU와 GPU에서 실행 가능
- 사용하기 쉬운 APO를 가지고 있어 딥러닝 모델의 프로토타입을 빠르게 만듬
- (컴퓨터 비전을 위한) 합성곱 신경망, (시퀀스 처리를 위한) 순환 신경망을 지원하면 이 둘을 자유롭게 조합 가능
- 다중 입력이나 다중 출력 모델, 층의 공유, 모델 공유 등 어떤 네트워크 구조도 만들 수 있음.
- └─> 적대적 생성 신경망(Generative Adversarial Network, GAN)부터 뉴럴 튜링 머신(Neural Turing Machine)까지 가능

## 딥러닝 컴퓨터 셋팅

- GPU 권장
- Unix 권장
- 이 책의 모든 코드 예제는 오픈 소스 노트북 파일로 제공
- [주소1](https://github.com/gilbutITbook/006975)
- [주소2](https://github.com/rickiepark/deep-learning-with-python-notebooks)

- 만약 GPU 환경을 설치하기 제한된다면 아래의 방법을 이용.
- 공식 EC2 딥러닝 [AMI](https://aws.amazon.com/amazon-ai/amis)를 사용해서 EC2에서 주피터 노트북으로 케라스 예제를 실행(부록C 참고)
- 위 방법은 클라우드에서 딥러닝을 수행하는 것으로, 장기적으로 이용하기에는 부적합하다. (요금을 내야하고 비싸다.)
- 2018년 중반 가격으로 시간당 0.9달라, GPU가격은 1000~1500달라. (당장은 EC2가 좋은 방법, 오랜 기간 사용할 것이라면 GPU 구매)
- GPU 구매시 'NVIDIA GPU'를 사야한다. (최신 딥러닝 프레임워크들은 NVIDIA 그래픽 카드에서만 작동한다.)

## 영화 리뷰 분류 : 이진 분류 예제

In [1]:
# 코드 3-1 IMDB 데이터셋 로드하기
from keras.datasets import imdb

# num_words = 10000 : 훈련데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다. - 적절한 크기의 벡터 데이터 얻음.
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

Using TensorFlow backend.


17465344/17464789 [==============================] - 3s 0us/step


In [11]:
print('train_data : ', train_data[0], '\n')
print('train_labels : ', test_labels[0], '\n')

# 가장 자주 등장하는 단어 1만 개로 제한 -> 인덱스 9999를 넘지 않음.
print(max([max(sequence) for sequence in train_data]))

train_data :  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 

train_labels :  0 

9999


In [13]:
word_index = imdb.get_word_index() # word_index는 단어와 정수 인덱스를 매핑한 딕셔너리.
reverse_word_index = dict( # 정수 인덱스와 단어를 매핑하도록 뒤집음.
    [(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join( # 리뷰 디코딩. 0, 1, 2는 '패딩', '문서 시작', '사전에 없음'을 위한 인덱스이므로 3을 뺌.
    [reverse_word_index.get(i-3, '?') for i in train_data[0]])

In [17]:
# 3-2 정수 시퀀스를 이진 행렬로 인코딩하기 (이해를 위해 직접 원-핫 인코딩.)
import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # results[i]에서 특정 인덱스의 위치를 1로 만듬.
    return results

# 벡터화
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [18]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [19]:
# 레이블은 쉽게 벡터로 바꿀 수 있음.
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

- 위 작업으로, 신경망에 주입할 데이터가 준비되었음
- 입력 데이터가 Vector고 레이블은 Scalar(1 또는 0)임
- 이런 문제에 잘 작동하는 네트워크 종류는 relu 활성화 함수를 사용한 완전 연결 층(즉 Dense(16, activation='relu'))을 그냥 쌓는 것.
- 여기에서 Dense(16)은 은닉 유닛(hidden unit)의 개수. 하나의 은닉 유닛은 층이 나타내는 표현 공간에서 하나의 차원이 됨.
- relu 활성화 함수를 사용한 Dense 층을 다음 텐서 연산을 연결하여 구현했다.
- ouput = relu(dot(W, input) + b)
- 16개의 은닉 유닛이 있다는 것은 가중치 행렬 W의 크기가 (input_dimension, 16)이라는 뜻.
- 입력 데이터와 W를 접곰하면 입력 데이터가 16차원으로 표현된 공간으로 투영.
- Dense 층을 쌓을 때 두 가지 중요한 결정
- 1. 얼마나 많은 층을 사용할 것인가?
- 2. 각 층에 얼마나 많은 은닉 유닛을 둘 것인가?
- 당분간은 다음과 같은 구조를 이용.
- 1. 16개의 은닉 유닛을 가진 2개의 은닉 층
- 2. 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층

In [21]:
# 코드 3-3 모델 정의하기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(16, activation = 'sigmoid'))

W0927 11:26:36.951156 15428 deprecation_wrapper.py:119] From C:\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0927 11:26:37.001326 15428 deprecation_wrapper.py:119] From C:\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [22]:
# 코드 3-4 모델 컴파일하기
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

W0927 13:38:35.088687 15428 deprecation_wrapper.py:119] From C:\Python\Python37\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0927 13:38:35.183432 15428 deprecation_wrapper.py:119] From C:\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:3657: The name tf.log is deprecated. Please use tf.math.log instead.

W0927 13:38:35.199389 15428 deprecation.py:323] From C:\Python\Python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
# 코드 3-5 옵티마이저 설정하기 (옵티마이저의 매개변수 변경시)
from keras import optimizers

model.compile(optimizer = optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [25]:
# 코드 3-6 손실과 측정을 함수 객체로 지정하기
from keras import losses
from keras import metrics

model.compile(optimizer = optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

## 뉴스 기사 분류 : 다중 분류 문제

## 주택 가격 예측 : 회귀 문제

In [2]:
784**(1/2)

28.0